<a href="https://colab.research.google.com/github/Fuohna/nlp-simple-chatbot-v1/blob/main/simple_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q -U "tensorflow==2.8.*"

In [116]:
import pandas as pd
import json
import string
import tensorflow as tf
from tensorflow import keras
import tensorflow_text as tf_text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Preprocessing

In [117]:
# leyzer_dev_url = 'https://raw.githubusercontent.com/cartesinus/leyzer/master/corpora/0.1.0/leyzer-dev-en-US-0.1.0.tsv'
leyzer_dev_url = "/content/drive/MyDrive/Colab Notebooks/NLP-Chatbot/train_dataset.csv"
leyzer_dev_df = pd.read_csv(leyzer_dev_url)

In [118]:
leyzer_df = leyzer_dev_df
# leyzer_df.drop(["bio"], axis = 1, inplace = True)
leyzer_df

,intent,utterance
0,TurnOff,switch the air conditioning off
1,TurnOff,turn the ac off
2,TurnOn,switch on the aircon
3,TurnOn,turn the aircon on
4,ChangeTemperature,change the minimum and maximum temperatures on...
...,...,...
1527,SelfAware,Can you prove you have a conscious
1528,SelfAware,Can you prove you are self-aware please
1529,SelfAware,Can you prove you are self aware please
1530,SelfAware,Can you prove you have a conscious please


In [119]:
leyzer_df['utterance'] = leyzer_df['utterance'].apply(lambda sequence: [ltrs.lower() for ltrs in sequence if ltrs not in string.punctuation])
leyzer_df

,intent,utterance
0,TurnOff,"[s, w, i, t, c, h, , t, h, e, , a, i, r, , ..."
1,TurnOff,"[t, u, r, n, , t, h, e, , a, c, , o, f, f]"
2,TurnOn,"[s, w, i, t, c, h, , o, n, , t, h, e, , a, ..."
3,TurnOn,"[t, u, r, n, , t, h, e, , a, i, r, c, o, n, ..."
4,ChangeTemperature,"[c, h, a, n, g, e, , t, h, e, , m, i, n, i, ..."
...,...,...
1527,SelfAware,"[c, a, n, , y, o, u, , p, r, o, v, e, , y, ..."
1528,SelfAware,"[c, a, n, , y, o, u, , p, r, o, v, e, , y, ..."
1529,SelfAware,"[c, a, n, , y, o, u, , p, r, o, v, e, , y, ..."
1530,SelfAware,"[c, a, n, , y, o, u, , p, r, o, v, e, , y, ..."


In [120]:
leyzer_df['utterance'] = leyzer_df['utterance'].apply(lambda word: ''.join(word))
leyzer_df

,intent,utterance
0,TurnOff,switch the air conditioning off
1,TurnOff,turn the ac off
2,TurnOn,switch on the aircon
3,TurnOn,turn the aircon on
4,ChangeTemperature,change the minimum and maximum temperatures on...
...,...,...
1527,SelfAware,can you prove you have a conscious
1528,SelfAware,can you prove you are selfaware please
1529,SelfAware,can you prove you are self aware please
1530,SelfAware,can you prove you have a conscious please


In [121]:
from nltk.tokenize import word_tokenize 
input_tokenized = list(leyzer_df['utterance'].apply(lambda sent: word_tokenize(sent)))
# print(input_tokenized)

In [122]:
col = leyzer_df[['intent']]
print("Total number of intents:", len(col.intent.unique()))
print(col.intent.unique())

Total number of intents: 202
['TurnOff' 'TurnOn' 'ChangeTemperature' 'SetTemperatureToValue'
 'SetTemperatureToValueOnDevice' 'SetTemperatureBetweenValues'
 'SetTemperatureOnDevice' 'GetHumidity' 'GetTemperature'
 'GetTemperatureFromDevice' 'OpenCalendar' 'CheckCalendarOnDate'
 'AddEventWithName' 'AddEventOnDateWithDate' 'NotifyOnEventStart'
 'NotNotifyOnEventStart' 'NotNotifyOnEventInLocation'
 'NotifyOnEventInLocation' 'OpenContacts' 'OpenMyContact'
 'EditContactWithName' 'EditContactWithNumber'
 'EditNumberInContactWithName'
 'EditNumberInContactWithNameToNumberAndToType' 'ShowContactWithEmail'
 'ShowContactWithName' 'ShowContactWithNameAndWithNumberAndWithType'
 'ShowContactWithNumber' 'ShowContactWithNumberAndWithEmail' 'OpenEmail'
 'ShowEmailFromSender' 'ShowEmailWithLabel' 'ShowEmailFromTime'
 'ShowEmailWithSubject' 'ShowEmailWithPriority'
 'SendEmailToAddressWithSubjectAndWithMessage'
 'SendEmailToAddressWithSubject' 'SendEmail' 'SendEmailToAddress'
 'ReplyToEmailFromAddress' '

In [123]:
def my_token(words):
  token = Tokenizer(1000)
  token = token.fit_on_texts(words)
  return token

In [124]:
tokenizer = Tokenizer(1000)
tokenizer.fit_on_texts(input_tokenized)
train_utr = tokenizer.texts_to_sequences(input_tokenized)
# train_utr = my_token(input_tokenized)
# print(train_utr)
features = pad_sequences(train_utr, maxlen = 100)
print(features[377])
print(len(features[0]))

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  71  10
  41 202  47  22  82   9  20  76 591 592]
100


In [125]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(features)
features = scaler.fit_transform(features)
print(features)

[[ 0.          0.          0.         ...  1.21956735  1.45170056
  -0.33655277]
 [ 0.          0.          0.         ... -0.68053396  1.45755631
  -0.33655277]
 [ 0.          0.          0.         ... -0.68593198 -0.61538209
   0.47174407]
 ...
 [ 0.          0.          0.         ...  2.93613615  3.31383166
  -0.44054418]
 [ 0.          0.          0.         ... -0.66433992  2.02556486
  -0.44054418]
 [ 0.          0.          0.         ... -0.24869276 -0.59781481
   1.19968391]]


In [137]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
unique_intent = list(leyzer_df['intent'])
# print(unique_intent)
token = Tokenizer(1000)
token.fit_on_texts(unique_intent)
intent_2_index = token.texts_to_sequences(unique_intent)
labels = OneHotEncoder(sparse=False).fit_transform(intent_2_index)
# print(labels)
label_2 = LabelEncoder().fit_transform(leyzer_df['intent'])
print(label_2)

[193 193 194 ... 121 121 121]


In [127]:
tokenizer.word_index

{'my': 1,
 'on': 2,
 'the': 3,
 'to': 4,
 'with': 5,
 'a': 6,
 'show': 7,
 'in': 8,
 'from': 9,
 'me': 10,
 'instagram': 11,
 'of': 12,
 'pictures': 13,
 'i': 14,
 'and': 15,
 'google': 16,
 'display': 17,
 'slack': 18,
 'this': 19,
 'youtube': 20,
 'you': 21,
 'new': 22,
 'get': 23,
 'number': 24,
 'restaurants': 25,
 'language': 26,
 '555': 27,
 'change': 28,
 'drive': 29,
 'play': 30,
 'matching': 31,
 'for': 32,
 'create': 33,
 'contact': 34,
 'post': 35,
 'list': 36,
 'open': 37,
 'at': 38,
 'files': 39,
 'name': 40,
 'when': 41,
 'facebook': 42,
 'spotify': 43,
 'emails': 44,
 'it': 45,
 'that': 46,
 'is': 47,
 'send': 48,
 'speaker': 49,
 'email': 50,
 'than': 51,
 'fitbit': 52,
 'as': 53,
 'images': 54,
 'an': 55,
 'check': 56,
 '1': 57,
 'by': 58,
 'twitter': 59,
 'picture': 60,
 'set': 61,
 'message': 62,
 'page': 63,
 'contacts': 64,
 'location': 65,
 'call': 66,
 'saying': 67,
 'weather': 68,
 'channels': 69,
 'what': 70,
 'notify': 71,
 'article': 72,
 'tweets': 73,
 'sear

In [128]:
vocabulary = len(tokenizer.word_index)
print("Total numbers of vocabulary:", vocabulary)

Total numbers of vocabulary: 1587


## Git_test_set

In [129]:
leyzer_test_df = pd.DataFrame(pd.read_csv("https://raw.githubusercontent.com/cartesinus/leyzer/master/corpora/0.1.0/leyzer-test-en-US-0.1.0.tsv", sep="\t"))
print(len(leyzer_test_df[['intent']].intent.unique()))
# print(leyzer_test_df[['intent']].intent.unique())

154


In [130]:
utr_test_tokenized = list(leyzer_test_df['utterance'].apply(lambda sent: word_tokenize(sent)))
# print(utr_test_tokenized)

test_tokenizer = Tokenizer(1000)
test_tokenizer.fit_on_texts(utr_test_tokenized)
test_utr = test_tokenizer.texts_to_sequences(utr_test_tokenized)
# print(test_utr)
test_sen = pad_sequences(test_utr, maxlen = 100)
print(test_sen[0])

t_scaler = StandardScaler().fit(test_sen)
test_sen = scaler.fit_transform(test_sen)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  23   3 327  17 328 329   2   1 106]


In [131]:
unique_test_intent = list(set(leyzer_test_df['intent']))
print(unique_test_intent)
t_token = Tokenizer(1000)
t_token.fit_on_texts(unique_test_intent)
t_intent_2_index = t_token.texts_to_sequences(unique_test_intent)
label_t = LabelEncoder().fit_transform(leyzer_test_df['intent'])
print(label_t)

['SearchByCategoryInLocation', 'CallEmergency', 'ShowPicturesWithHashtag', 'PostPictureWithUrlAndWithCaption', 'RandomPage', 'OpenEngine', 'SetTemperatureOnDevice', 'OpenMyContact', 'CheckMessagesInChannel', 'ShowChannelWithName', 'ScrollDown', 'MoonphaseInLocationOnDate', 'GetTemperature', 'GetHumidity', 'SendEmailToAddress', 'CheckLastMessages', 'SendMessageToUser', 'SearchByQueryInLocation', 'OpenCalendar', 'OpenTwitter', 'SendMessageToChannel', 'OpenGdrive', 'SendPictureToChannelWithCaption', 'NotNotifyOnEventInLocation', 'TurnOnOrOff', 'UnfollowUser', 'ShowFilesFromTime', 'OpenWikipedia', 'ShowContactWithNumberAndWithEmail', 'NotifyOnNewFromFollowing', 'SendEmailToAddressWithSubjectAndWithMessage', 'ShowNews', 'WeatherTomorrowInLocation', 'SearchRestaurant', 'SetTemperatureBetweenValues', 'FindQuery', 'SendEmail', 'DetectLanguage', 'ShowPicturesWithLocation', 'NotNotifyOnEventStart', 'NotifyOnEventInLocation', 'OpenFileWithName', 'ShowSubscribedChannels', 'PostPictureWithCaption',

# Model

In [132]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [133]:
X_train = pd.DataFrame(data = features)
# print(X_train)
y_train = pd.DataFrame(data = label_2)
# print(y_train)
# X_test = pd.DataFrame(data = test_sen)
# y_test = pd.DataFrame(data = label_t)

# X_train, X_test, y_train, y_test = train_test_split(features, label_2, test_size = 0.2, random_state = 42)
clf = SVC(kernel='rbf', gamma=0.01, C=1000)
clf.fit(X_train, y_train)

# prediction = clf.predict(X_test)
# print(accuracy_score(y_test, prediction))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1000, gamma=0.01)

In [134]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier()
clf.fit(X_train, y_train);
# prediction = clf.predict(X_test);
# print(accuracy_score(y_test, prediction))

<ipython-input-134-a6a4fc09359f>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train);


In [135]:
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def _predict(input):
  input = input.lower()
  print(input)
  input = word_tokenize(input)
  test_ls = Tokenizer().texts_to_sequences(input)

  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = pad_sequences(test_ls, maxlen = 100)
  
  pred = clf.predict(x)
  return pred

In [136]:
text = "Can you help me?"
_predict(text)

can you help me?


array([148])